Imports


In [1]:
from collections import defaultdict
import Reddit_MDA
import os
import torch
import flair
import concurrent.futures
from concurrent.futures import ThreadPoolExecutor
import logging
import queue
import random
import threading
from threading import Thread
import time


c:\Users\gusta\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2023-05-26 15:07:37,899 loading file D:/example-pos/RedditTaggerFinal150.pt
2023-05-26 15:08:16,620 SequenceTagger predicts: Dictionary with 49 tags: <unk>, O, LS, NN, VBG, VBZ, DT, RB, JJ, WDT, VBN, IN, CC, ., NNS, VBP, ,, TO, -LRB-, NNP, -RRB-, RBR, JJR, PRP, CD, VB, MD, PRP$, POS, RBS, :, JJS, NNPS, VBD, RP, WRB, WP, SYM, EX, FW, UH, -LSB-, -RSB-, ``, '', PDT, WP$, <START>, <STOP>
0:00:42.756112


Enabling and checking cuda device

In [3]:
device = None
if torch.cuda.is_available():
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu')
torch.cuda.set_per_process_memory_fraction(0.9, 0)
torch.cuda.empty_cache()
total_memory = torch.cuda.get_device_properties(0).total_memory
# less than 0.5 will be ok:
tmp_tensor = torch.empty(int(total_memory * 0.8), dtype=torch.int8, device='cuda')
del tmp_tensor
torch.cuda.empty_cache()
# this allocation will raise a OOM:
torch.empty(total_memory // 2, dtype=torch.int8, device='cuda')


flair.device = torch.device('cuda:0')
print(flair.device)


cuda:0


Opening files and creating empty dictionary

In [4]:
sents = {}

with open("sample_sentences.txt", errors = "ignore") as f:
    for line in f:
        sents[line.split("\t")[0]] = line.split("\t")[1].strip("\n")


feats = ['vpast_001', 'vpresperfect_002a', 'vpastperfect_002b', 'vpresent_003', 'advplace_004', 'advtime_position_005a', 'advtime_durfreq_005b', 'profirpers_006', 'prosecpers_007', 'prothirdper_008', 'proit_009', 'prodemons_010', 'proindef_011', 'pverbdo_012', 'whquest_013', 'nominalis_014', 'gerund_015', 'nouns_016', 'passagentl_017', 'passby_018', 'mainvbe_019', 'exthere_020', 'thatvcom_021', 'thatacom_022', 'whclause_023', 'vinfinitive_024', 'vpresentpart_025', 'vpastpart_026', 'vpastwhiz_027', 'vpresentwhiz_028', 'thatresub_029', 'thatreobj_030', 'whresub_031', 'whreobj_032', 'whrepied_033', 'sentencere_034', 'advsubcause_035', 'advsubconc_036', 'advsubcond_037', 'advsubother_038', 'prepositions_039', 'adjattr_040', 'adjpred_041', 'adverbs_042', 'conjuncts_045', 'downtoners_046', 'hedges_047', 'amplifiers_048', 'discpart_050', 'demonstr_051', 'modalsposs_052', 'modalsness_053', 'modalspred_054', 'vpublic_055', 'vprivate_056', 'vsuasive_057', 'vseemappear_058', 'contractions_059', 'thatdel_060', 'strandprep_061', 'vsplitinf_062', 'vsplitaux_063', 'coordphras_064', 'coordnonp_065', 'negsyn_066', 'negana_067', 'hashtag_201', 'link_202', 'interlink_203', 'caps_204', 'vimperative_205', 'lengthening_206', 'emoticons_207', 'question_208', 'exclamation_209', 'comparatives_syn_212', 'superlatives_syn_213', 'comparatives_ana_214', 'superlatives_ana_215', 'reddit_vocab_216', 'vprogressive_217']

p = open("Feature_coding_discrepancies_flair2.tsv", "w")
p.write("Feature\tsentence\tManualCount\tAutoCount\n")
p2 = open("Feature_coding_accuracies_flair2.tsv", "w")
p2.write("Feature\tAccuracy\tTrue_positives\tFalse_positives\tTrue_negatives\tFalse_negatives\tPrecision\tRecall\tFScore\n")


103

Iterate over files and generate output

In [5]:
def script(feats):    
    for feat in feats:
        sent_count = 0
        error_count = 0
        true_pos = 0
        false_pos = 0
        true_neg = 0
        false_neg = 0
        if os.path.isfile("Manual_coding_files\manual_coding_"+ feat + "_corrected.txt"):
            filstring = "_corrected"
        else:
            filstring = ""

        with open("Manual_coding_files\manual_coding_"+ feat + filstring + ".txt") as f:
            for line in f:
                sent_count += 1
                numid = line.split("\t")[0]
                manual = int(line.strip("\n").split("\t")[1])
                auto = Reddit_MDA.process_sent(sents[numid], feat)
                if manual!= auto:
                    error_count += 1
                    p.write("\t".join([feat, sents[numid].strip("'").strip('"'), str(manual), str(auto)]) + "\n")
                if manual > 0 and auto > 0:
                    true_pos += 1
                elif manual == 0 and auto > 0:
                    false_pos += 1
                elif manual == 0 and auto == 0:
                    true_neg += 1
                else:   
                    false_neg += 1 
        if true_pos > 0:
            precision = true_pos/(true_pos+false_pos)
            recall = true_pos/(true_pos+false_neg)
            Fscore = 2*((precision*recall)/(precision+recall))
            p2.write(feat + "\t" + str((sent_count-error_count)/sent_count) + "\t" + "\t".join([str(true_pos), str(false_pos), str(true_neg), str(false_neg), str(precision), str(recall), str(Fscore)]) + "\n")
        else:
            p2.write(feat + "\t" + str((sent_count-error_count)/sent_count) + "\t" + "\t".join([str(true_pos), str(false_pos), str(true_neg), str(false_neg), "xxx", "xxx", "xxx"]) + "\n")

    p.close()
    p2.close()

multi-threading

In [16]:
# create a thread pool
with ThreadPoolExecutor(max_workers=10) as pool:
	# submit tasks and get results
	# ...
	# automatically shutdown the thread pool...
# the pool is shutdown at this point
    futures = [pool.submit(script, feats)]
    futures

2023-05-26 19:47:23,557 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2023-05-26 19:47:25,203 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2023-05-26 19:47:27,981 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2023-05-26 19:47:29,642 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2023-05-26 19:47:33,794 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2023-05-26 19:47:35,828 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2023-05-26 19:47:37,633 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2023-05-26 19:47:39,294 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2023-05-26 19:47:43,431 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2023-05-26 19:47:45,484 Warning: An empty Sentence was 